<a href="https://colab.research.google.com/github/HUJameson/Colab/blob/main/aillm_0102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install readability-lxml

In [ ]:
import requests
from readability import Document
import openai


def get_webpage_content(url):
    # 发送GET请求获取网页内容
    response = requests.get(url)
    html = response.content

    # 使用readability库提取网页正文内容
    doc = Document(html)
    content = doc.summary()

    return content

def generate_summary(text):
    # 调用Open AI的API生成摘要
    response = openai.Completion.create(
        engine="davinci",
        prompt=text,
        max_tokens=100,
        temperature=0.7,
        n=1,
        stop=None
    )
    summary = response.choices[0].text.strip()

    return summary

# 输入URL
url = "https://zhuanlan.zhihu.com/p/611905895"

# 获取网页内容
#content = get_webpage_content(url)
content="""当你走进这欢乐场
背上所有的梦与想
各色的脸上各色的妆
没人记得你的模样

三巡酒过你在角落
固执的唱着苦涩的歌
听它在喧嚣里被淹没
你拿起酒杯对自己说

一杯敬朝阳 一杯敬月光
唤醒我的向往 温柔了寒窗
于是可以不回头地逆风飞翔
不怕心头有雨 眼底有霜

一杯敬故乡 一杯敬远方
守着我的善良 催着我成长
所以南北的路从此不再漫长
灵魂不再无处安放

躁动不安的座上客
自以为是地表演着
伪装着 舞蹈着 疲惫着
你拿起酒杯对自己说
一杯敬朝阳 一杯敬月光
唤醒我的向往 温柔了寒窗
于是可以不回头地逆风飞翔
不怕心头有雨 眼底有霜

一杯敬故乡 一杯敬远方
守着我的善良 催着我成长
所以南北的路从此不再漫长
灵魂不再无处安放

一杯敬明天 一杯敬过往
支撑我的身体 厚重了肩膀
虽然从不相信所谓山高水长
人生苦短何必念念不忘

一杯敬自由 一杯敬死亡
宽恕我的平凡 驱散了迷惘
好吧天亮之后总是潦草离场
清醒的人最荒唐
清醒的人最荒唐"""

# 生成摘要
summary = generate_summary(content)

# 打印摘要
print("摘要：", summary)

In [ ]:
import openai
from openai.embeddings_utils import cosine_similarity, get_embedding

# 选择使用最小的ada模型
EMBEDDING_MODEL = "text-embedding-ada-002"

# 获取"好评"和"差评"的
positive_review = get_embedding("好评")
negative_review = get_embedding("差评")

positive_example = get_embedding("买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质")
negative_example = get_embedding("降价厉害，保价不合理，不推荐")

def get_score(sample_embedding):
  return cosine_similarity(sample_embedding, positive_review) - cosine_similarity(sample_embedding, negative_review)

positive_score = get_score(positive_example)
negative_score = get_score(negative_example)

print("好评例子的评分 : %f" % (positive_score))
print("差评例子的评分 : %f" % (negative_score))

In [ ]:
good_restraurant = get_embedding("这家餐馆太好吃了，一点都不糟糕")
bad_restraurant = get_embedding("这家餐馆太糟糕了，一点都不好吃")

good_score = get_score(good_restraurant)
bad_score = get_score(bad_restraurant)
print("好评餐馆的评分 : %f" % (good_score))
print("差评餐馆的评分 : %f" % (bad_score))

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)

# convert 5-star rating to binary sentiment
df = df[df.Score != 3]
df["sentiment"] = df.Score.replace({1: "negative", 2: "negative", 4: "positive", 5: "positive"})

from sklearn.metrics import PrecisionRecallDisplay

def evaluate_embeddings_approach(
    labels = ['negative', 'positive'],
    model = EMBEDDING_MODEL,
):
    label_embeddings = [get_embedding(label, engine=model) for label in labels]

    def label_score(review_embedding, label_embeddings):
        return cosine_similarity(review_embedding, label_embeddings[1]) - cosine_similarity(review_embedding, label_embeddings[0])

    probas = df["embedding"].apply(lambda x: label_score(x, label_embeddings))
    preds = probas.apply(lambda x: 'positive' if x>0 else 'negative')

    report = classification_report(df.sentiment, preds)
    print(report)

    display = PrecisionRecallDisplay.from_predictions(df.sentiment, probas, pos_label='positive')
    _ = display.ax_.set_title("2-class Precision-Recall curve")

evaluate_embeddings_approach(labels=['An Amazon review with a negative sentiment.', 'An Amazon review with a positive sentiment.'])